In [ ]:
import vtk
from vtk.util.numpy_support import numpy_to_vtk
import numpy as np
from IPython.display import Image
import IPython.display as display
from matplotlib import pyplot as plt
import pandas as pd
import notebookrender
import ipywidgets

## Load the Fourier basis

In [ ]:
U = np.loadtxt('U-30.csv', dtype=np.complex128)

plt.figure(figsize=(18,6))
plt.plot(np.real(U[:,1]))
plt.show()

## Load a sample data set

In [ ]:
path = '../Variationsanalyse/data/sto_2/run__{:05d}/m_1/vtk_Export/Assembly/Assembly_6.vtu'
reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName(path.format(1))
reader.Update()
data = reader.GetOutput()

## Display the first 10 Fourier basis

In [ ]:
for i in range(0,10):
    U_res = numpy_to_vtk(np.real(U)[:,i])
    U_res.SetName('U_res')
    data.GetPointData().AddArray(U_res)
    re = notebookrender.rendering(
        data, 
        width=800, 
        height=600, 
        pos=[200.0, 0.0, 4500.0], 
        foc=[200.0, 0.0, 1000.0], 
        zoom=1.3, 
        scalarRange=U_res.GetRange(),
        colorBarCoordinate=(0.7,0.1)
    )
    data.GetPointData().RemoveArray('U_res')

    display.display(
        ipywidgets.HBox([
            ipywidgets.Text(str(i), disabled=True),
            re
        ], layout=ipywidgets.Layout(height='300px'))
    )

## Load the training data set

In [ ]:
loadPath = '../Variationsanalyse/data/sto_2/run__0{:04d}/m_1/assembly_6.csv'
numberOfTests = 793
n = 188559
df = np.zeros((numberOfTests, n, 3), dtype=np.float32)
for testId in range(0, numberOfTests):
    df_temp = pd.read_csv(loadPath.format(testId+1), sep=' ', header=0, dtype=np.float32)
    df[testId,:] = np.reshape(df_temp, (-1,3))
    
print(df.shape)

## Spectrum computation

In [ ]:
S = np.real(U.transpose()) @ df
print(S.shape)

## Boxplots for the spectrum of all parts

In [ ]:
plt.figure(figsize=(18,10))
ax1 = plt.subplot(3,1,1)
plt.boxplot(np.real(S[...,0]))
plt.setp(ax1.get_xticklabels(), visible=False)
plt.ylabel('X')
plt.grid()

ax2 = plt.subplot(3,1,2)
plt.boxplot(np.real(S[...,1]))
plt.setp(ax2.get_xticklabels(), visible=False)
plt.ylabel('Y')
plt.grid()

ax3 = plt.subplot(3,1,3)
plt.boxplot(np.real(S[...,2]))
plt.xlabel(r'$\omega$')
plt.ylabel('Z')
plt.grid()

plt.subplots_adjust(hspace=0.05)
plt.show()

## Variance of all spectri

In [ ]:
S_std = np.std(np.real(S), axis=0)
x = np.arange(30)
width = 0.25

plt.figure(figsize=(18,6))
ax = plt.gca()
ax.bar( x-width, S_std[:30,0], width, label='X' )
ax.bar( x, S_std[:30,1], width, label='Y' )
ax.bar( x+width, S_std[:30,2], width, label='Z' )

ax.grid(axis='y')
ax.legend()
ax.set_xticks(np.arange(0,30,2))
plt.xlabel(r'$\omega$')
plt.ylabel(r'$var(\Omega_V)$')

plt.tight_layout()

plt.show()